In [1]:
import pandas as pd
from omg.checker import Checker
from omg.consistency import IdentifierConsistencyAssertion, TimeConsistencyAssertion

In [2]:
# In this example, we assume the data is generated from some deep learning method
# We'll assume the dataframe is already populated

df = pd.DataFrame(
    [[0, 1, 'Christi Paul', 'Female', 'Black'],
     [1, 1, 'Christi Paul', 'Female', 'Brown'],
     [3, 1, 'Poppy Harlow', 'Male', 'Black']],
    columns=['frame', 'scene_idenfier', 'name', 'gender', 'hair_color']
)

In [3]:
df

,frame,scene_idenfier,name,gender,hair_color
0,0,1,Christi Paul,Female,Black
1,1,1,Christi Paul,Female,Brown
2,3,1,Poppy Harlow,Male,Black


In [4]:
# The prediction function here simply returns the predicted values
# This would normally call a deep learning serving method

def prediction_function(df):
    return df[['name', 'gender', 'hair_color']]

In [5]:
# Define the assertions and register them

gender_consistency = IdentifierConsistencyAssertion('name', 'gender')
hair_color_consistency = IdentifierConsistencyAssertion('scene_idenfier', 'hair_color')
time_consistency = TimeConsistencyAssertion('scene_idenfier', 'frame')

checker = Checker(name='Consistency checker')
checker.register_assertion(gender_consistency.get_assertion(), gender_consistency.get_name())
checker.register_assertion(hair_color_consistency.get_assertion(), hair_color_consistency.get_name())
checker.register_assertion(time_consistency.get_assertion(), time_consistency.get_name())

In [6]:
# Run the prediction and check for errors

pred_fn = checker.wrap(prediction_function)
outs = pred_fn(df)

Assertion Identifier consistency name, gender failed on [[0, 2]]
   frame  scene_idenfier          name  gender hair_color
0      0               1  Christi Paul  Female      Black
2      3               1  Poppy Harlow    Male      Black            name  gender hair_color
0  Christi Paul  Female      Black
2  Poppy Harlow    Male      Black
Assertion Identifier consistency scene_idenfier, hair_color failed on [[0, 1]]
   frame  scene_idenfier          name  gender hair_color
0      0               1  Christi Paul  Female      Black
1      1               1  Christi Paul  Female      Brown            name  gender hair_color
0  Christi Paul  Female      Black
1  Christi Paul  Female      Brown
Assertion Time consistency scene_idenfier, 3 failed on [0, 0]
frame                        0
scene_idenfier               1
name              Christi Paul
gender                  Female
hair_color               Black
Name: 0, dtype: object name          Christi Paul
gender              Female
hair